# Coeficiente de Rayleigh  

<br />

$$ \omega \approx  \sqrt{\frac{\displaystyle \sum_{i=1}^n F_i \cdot D_i}{  \displaystyle \sum_{i=1}^n M_i \cdot D^2_i}} \rightarrow T =  2 \pi { \sqrt{\frac{\displaystyle \sum_{i=1}^n W_i \cdot D^2_i}{g  \displaystyle \sum_{i=1}^n F_i \cdot D_i}}}  $$

## 1. Librerías

In [1]:
import numpy as np
np.set_printoptions(formatter = {'float': lambda x: '{:.4f}'.format(x)})

import pandas as pd
pd.options.display.float_format = '{:,.3f}'.format

from matplotlib import pyplot as plt


## 2. Grados de Libertad

In [2]:
while True:
    try:
        gdl = int(input('* Ingrese el número de grados de libertad: '))
        break
    except ValueError:
        print('! Ingrese un número de GDL válido.\n')

print(f'* El modelo es de {gdl} GDL.')

* El modelo es de 3 GDL.


## 3. Rigideces de Entrepiso

In [3]:
k = np.empty(gdl)

for i in range(gdl):
    while True:
        try:
            k[i] = float(input(f'* Rigidez K{i + 1} (ton/m): '))
            print(f'\n> Rigidez K{i + 1} = {k[i]} ton/m')
            break
        except ValueError:
            print(f'\n! Ingrese un valor de K{i + 1} válido.')


> Rigidez K1 = 25000.0 ton/m

> Rigidez K2 = 21000.0 ton/m

> Rigidez K3 = 17000.0 ton/m


## 4. Masas Sísmicas

In [4]:
m = np.empty(gdl)

for i in range(gdl):
    while True:
        try:
            m[i] = float(input(f'* Masa M{i + 1} (ton-s2/m): '))
            print(f'\n> Masa M{i + 1} = {m[i]} ton-s2/m')
            break
        except ValueError:
            print(f'\n! Ingrese un valor de M{i + 1} válido.')


> Masa M1 = 15.0 ton-s2/m

> Masa M2 = 13.0 ton-s2/m

> Masa M3 = 11.0 ton-s2/m


## 5. Fuerzas Asumidas

In [5]:
F = np.empty(gdl)

for i in range(gdl):
    while True:
        try:
            F[i] = float(input(f'* Fuerza F{i + 1} (ton): '))
            print(f'\n> Fuerza F{i + 1} = {F[i]:.2f} ton')
            break
        except ValueError:
            print(f'\n! Ingrese un valor de F{i + 1} válido.')


> Fuerza F1 = 10000.00 ton

> Fuerza F2 = 20000.00 ton

> Fuerza F3 = 30000.00 ton


## 6. Tabla de Entrada

In [6]:
tabla = pd.DataFrame({'Mi (ton-s2/m)': m[::-1], 'Ki (ton/m)': k[::-1], 'Fi (ton)': F[::-1]})
tabla.index = np.arange(gdl, 0, -1)

tabla

,Mi (ton-s2/m),Ki (ton/m),Fi (ton)
3,11.000,"17,000.000","30,000.000"
2,13.000,"21,000.000","20,000.000"
1,15.000,"25,000.000","10,000.000"


## 6. Cálculos Tabulares

### 6.1 Cortante y Desplazamientos

In [7]:
# Cortante en entrepiso
tabla['Vi (ton)'] = tabla['Fi (ton)'].cumsum()

# Desplazamientos relativos
tabla['Δi (m)'] = tabla['Vi (ton)'] / tabla['Ki (ton/m)']

# Desplazamientos absolutos
tabla['Di (m)'] = tabla['Δi (m)'][::-1].cumsum()

tabla



,Mi (ton-s2/m),Ki (ton/m),Fi (ton),Vi (ton),Δi (m),Di (m)
3,11.000,"17,000.000","30,000.000","30,000.000",1.765,6.546
2,13.000,"21,000.000","20,000.000","50,000.000",2.381,4.781
1,15.000,"25,000.000","10,000.000","60,000.000",2.400,2.400


### 6.2 Valores Intermedios

In [8]:
tabla['Mi * Di2 (ton-s2-m)'] = tabla['Mi (ton-s2/m)'] * np.power(tabla['Di (m)'], 2)
tabla['Fi * Di (ton-m)'] = tabla['Fi (ton)'] * tabla['Di (m)']

tabla

,Mi (ton-s2/m),Ki (ton/m),Fi (ton),Vi (ton),Δi (m),Di (m),Mi * Di2 (ton-s2-m),Fi * Di (ton-m)
3,11.000,"17,000.000","30,000.000","30,000.000",1.765,6.546,471.302,"196,369.748"
2,13.000,"21,000.000","20,000.000","50,000.000",2.381,4.781,297.148,"95,619.048"
1,15.000,"25,000.000","10,000.000","60,000.000",2.400,2.400,86.400,"24,000.000"


## 7. Resultados

### 7.1 Sumatorias de Fin de Tabla

In [9]:
sum_mi_di2 = tabla['Mi * Di2 (ton-s2-m)'].sum()
sum_fi_di = tabla['Fi * Di (ton-m)'].sum()

print(f'* Σ Mi Di2 = {sum_mi_di2:.3f} ton-s2-m\n')
print(f'* Σ Fi Di = {sum_fi_di:.3f} ton-m')

* Σ Mi Di2 = 854.850 ton-s2-m

* Σ Fi Di = 315988.796 ton-m


### 7.2 Frecuencias y Período

In [10]:
# Frecuencia circular
w = np.sqrt(sum_fi_di / sum_mi_di2)

print(f'- Frecuencia circular: {w:.3f} rad/s\n')

# Frecuencia natural
f = w / (2 * np.pi)

print(f'- Frecuencia natural: {f:.3f} Hz\n')

# Período de vibración
T = 1 / f

print(f'- Período de vibración: {T:.3f} s')

- Frecuencia circular: 19.226 rad/s

- Frecuencia natural: 3.060 Hz

- Período de vibración: 0.327 s
